Celda 1: Instalación de librerías



In [1]:
# 1. Instalamos las librerías necesarias
# Gradio: Para la interfaz visual
# Deep-translator: Para traducir español -> inglés (ya que entrenamos con datos en inglés)
!pip install -q gradio deep-translator transformers torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 1.5 MB/s eta 0:00:00


Celda 2: Carga del Modelo desde Drive
La celda conecta con Google Drive y carga el modelo que entrenamos en el Notebook 2.

In [2]:
import os
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from google.colab import drive

# 2. Montar Google Drive
# Si ya está montado, esto no hará nada, si no, pedirá permisos.
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

# --- CONFIGURACIÓN ---
# Asegúrate de que esta ruta sea EXACTAMENTE donde guardaste el modelo en el Notebook 2
MODEL_DIR = "/content/drive/MyDrive/proyecto_ia/modelos/sentimiento_xlmroberta_v1"

print(f"🔄 Cargando modelo desde: {MODEL_DIR} ...")

try:
    # Cargar Tokenizer y Modelo
    tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR)
    model = AutoModelForSequenceClassification.from_pretrained(MODEL_DIR)

    # Configurar dispositivo (GPU si está disponible, si no CPU)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    print(f"✅ Modelo cargado exitosamente. Corriendo en: {device}")

except Exception as e:
    print(f"❌ ERROR CRÍTICO: No se pudo cargar el modelo.")
    print(f"Detalle: {e}")
    print("\n💡 SOLUCIÓN: Verifica que la carpeta '{MODEL_DIR}' exista en tu Drive y contenga los archivos config.json, model.safetensors, etc. Asegúrate de que los permisos de acceso a Google Drive sean correctos y que la ruta sea accesible.") # Added more specific advice here


Mounted at /content/drive
🔄 Cargando modelo desde: /content/drive/MyDrive/proyecto_ia/modelos/sentimiento_xlmroberta_v1 ...


The tokenizer you are loading from '/content/drive/MyDrive/proyecto_ia/modelos/sentimiento_xlmroberta_v1' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.


✅ Modelo cargado exitosamente. Corriendo en: cuda


Celda 3: Lógica y App (Interfaz Gráfica)
Celda final. Al ejecutarla, aparecerá la aplicación interactiva.

In [ ]:
import gradio as gr
import torch.nn.functional as F
from deep_translator import GoogleTranslator

# --- CONFIGURACIÓN DE LA APP ---

# Inicializar traductor (Español -> Inglés)
translator = GoogleTranslator(source='auto', target='en')

# Mapeo de etiquetas (0, 1, 2) a texto legible
labels_map = {
    0: "Negativo 😠",
    1: "Neutro 😐",
    2: "Positivo 😃"
}

def predecir_sentimiento(comentario):
    """
    Función que recibe el texto del usuario, lo traduce,
    lo pasa por el modelo y devuelve las probabilidades.
    """
    if not comentario:
        return None

    try:
        # 1. Traducir al inglés (necesario por el entrenamiento previo)
        texto_en = translator.translate(comentario)
    except Exception as e:
        return {"Error de conexión/traducción": 0.0}

    # 2. Tokenizar
    inputs = tokenizer(
        texto_en,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=512
    ).to(device)

    # 3. Inferencia (Predicción)
    model.eval() # Modo evaluación
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits

    # 4. Convertir logits a probabilidades (0 a 1)
    probs = F.softmax(logits, dim=1).squeeze().cpu().numpy()

    # 5. Formatear para Gradio (Diccionario {Etiqueta: Probabilidad})
    # Esto le dice a Gradio qué barras dibujar
    resultado_dict = {
        labels_map[0]: float(probs[0]),
        labels_map[1]: float(probs[1]),
        labels_map[2]: float(probs[2])
    }

    return resultado_dict

# --- CREACIÓN DE LA INTERFAZ ---

interface = gr.Interface(
    fn=predecir_sentimiento, # La función cerebro
    inputs=gr.Textbox(
        lines=3,
        placeholder="Escribe aquí tu reseña (ej: Me encantó el producto...)",
        label="Tu Reseña"
    ),
    outputs=gr.Label(num_top_classes=3, label="Probabilidades"), # Gráfico de barras
    title="🛒 Analizador de Sentimientos (Amazon Demo)",
    description="""
    **Instrucciones:** Escribe una opinión sobre un producto en Español.
    El sistema traducirá internamente y analizará si es Positiva, Negativa o Neutra.
    """,
    examples=[
        ["El producto llegó roto y la caja estaba abierta. Pésimo servicio."],
        ["Es increíble, la calidad es superior a lo que esperaba. Recomendado."],
        ["Cumple su función, pero tardó un poco en llegar. Es un producto normal."],
        ["No estoy seguro si me gusta, tiene cosas buenas y malas."]
    ],
    theme="soft"
)

# Lanzar la app
# debug=True ayuda a ver errores si algo falla
# share=True genera un link público temporal (útil para mostrar a otros)
interface.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://a0dbcc3959416dc849.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
